In [1]:
import pandas as pd
import numpy as np
import requests

In [2]:
portfolio = pd.read_csv('portfolio.csv')
portfolio['Дата'] = portfolio['Дата'].astype('datetime64[ns]')
transactions = pd.read_csv('transactions.csv').drop_duplicates(['Дата заключения', 'Время заключения', 'Статус'])
transactions['Дата заключения'] = pd.to_datetime(transactions['Дата заключения'] + ' ' + transactions['Время заключения'], dayfirst=True)
transactions['Дата расчетов'] = pd.to_datetime(transactions['Дата расчетов'], dayfirst=True)
transactions = transactions.drop('Время заключения', axis=1).dropna(axis=1).sort_values('Дата заключения').reset_index(drop=True)

In [3]:
transactions

,Дата заключения,Дата расчетов,Наименование,Код,Валюта,Вид,Количество,Цена,Сумма,НКД,Комиссия Брокера,Комиссия Биржи,Номер сделки,Статус
0,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.0000,1633.0,0.0,0.98,0.15,3943134134,И
1,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.0000,1633.0,0.0,0.98,0.15,3943134134,О
2,2021-05-17 10:21:16,2021-05-19,SBSPETF,SBSP,RUB,Покупка,1,1633.0000,1633.0,0.0,0.98,0.15,3943134134,ЗО
3,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.0000,5615.0,0.0,3.37,0.52,3943142919,О
4,2021-05-17 10:22:48,2021-05-19,FXUSETF,FXUS,RUB,Покупка,1,5615.0000,5615.0,0.0,3.37,0.52,3943142919,ЗО
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
276,2023-08-14 19:06:28,2023-08-15,СевСт-ао,CHMF,RUB,Покупка,1,1321.8000,1321.8,0.0,0.79,0.40,8088131206,И
277,2023-08-14 19:07:03,2023-08-15,Системаао,AFKS,RUB,Покупка,100,18.1070,1810.7,0.0,1.09,0.54,8088134689,ЗО
278,2023-08-14 19:07:03,2023-08-15,Системаао,AFKS,RUB,Покупка,100,18.1070,1810.7,0.0,1.09,0.54,8088134689,И
279,2023-08-14 19:07:50,2023-08-15,ИнтерРАОао,IRAO,RUB,Покупка,400,4.1505,1660.2,0.0,1.00,0.50,8088139159,ЗО


In [4]:
transactions_executed = transactions[transactions['Статус'] == 'И']

In [5]:
ticker_list = list(transactions_executed['Код'].unique())
share_split_dict = {
    'FXGD': ['2022-02-17', 10],
    'SBMX': ['2021-06-09', 100],
    'FXUS': ['2022-01-24', 100],
    'FXRL': ['2021-11-24', 100],
    'FXRU': ['2022-02-17', 10],
    'FXDE': ['2021-12-15', 100]
}
replacement_dict = {
    'VTBE': 'RSHE',
    'RU000A101FA1': 'SU25084RMFS3'
}

In [6]:
def fix_split():
    ticker_list_copy = ticker_list.copy()
    for ticker in ticker_list_copy:
        if ticker == 'RU000A101FA1':
            new_ticker = 'SU25084RMFS3'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
        if ticker == 'VTBE':
            new_ticker = 'RSHE'
            ticker_list.remove(ticker)
            ticker_list.append(new_ticker)
            transactions_executed.loc[transactions['Код'] == ticker, 'Код'] = new_ticker
    for ticker in ticker_list:
        if ticker in share_split_dict:
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] = \
            transactions_executed.loc[(transactions_executed['Код'] == ticker) & (transactions_executed['Дата заключения'] < share_split_dict[ticker][0]), 'Количество'] * \
            share_split_dict[ticker][1]
fix_split()

In [7]:
def get_share_price_dict(tickers: list):
    share_price_dict = {}
    for ticker in tickers:
        if ticker in ['SBMX', 'AFKS']:
            round_numb = 3
        elif ticker in ['IRAO']:
            round_numb = 4
        else:
            round_numb = 2
        share = transactions_executed[transactions_executed['Код'] == ticker]
        share_price = ((share['Сумма'].sum() + share['Комиссия Брокера'].sum() + share['Комиссия Биржи'].sum()) / 
                            (share.loc[share['Вид'] == 'Покупка', 'Количество'].sum() - share.loc[share['Вид'] == 'Продажа', 'Количество'].sum())).round(round_numb)
        share_price_dict[ticker] = share_price
    return share_price_dict

In [8]:
share_price_dict = get_share_price_dict(ticker_list)

In [9]:
def get_share_amount_dict(tickers: list):
    share_amount_dict = {}
    for ticker in tickers:
        share_amount_dict[ticker] = transactions_executed.loc[transactions_executed['Код'] == ticker, 'Количество'].sum()
    return share_amount_dict

In [10]:
share_amount_dict = get_share_amount_dict(ticker_list)

In [11]:
def get_trading_dict():
    trading_dict = {}
    trading_mode_list = ['TQBR', 'TQTF', 'TQCB', 'TQOB']
    for id_trading in trading_mode_list:
        stocks = 'shares'
        if id_trading in ['TQOB', 'TQCB']:
            stocks = 'bonds'
        url = (f"https://iss.moex.com/iss/engines/stock/markets/{stocks}/boards/{id_trading}/"
            f"securities.csv?iss.meta=off&iss.only=marketdata&marketdata.columns=SECID,LAST")
        csv_text = requests.get(url).text.split('\n')
        trading_dict[id_trading] = csv_text
    return trading_dict

In [12]:
def get_last_prices_dict(tickers: list):
    last_prices_dict = {}
    trading_dict = get_trading_dict()
    for ticker in tickers:
        if ticker == 'RU000A101FA1':
            ticker =  'SU25084RMFS3'
        if ticker == 'VTBE':
            ticker = 'RSHE'
        for value in trading_dict.values():
            for line in value:
                line = line.split(';')
                if ticker in line:
                    last_prices_dict[ticker] = line[1]
    return last_prices_dict

In [13]:
last_prices_dict = get_last_prices_dict(ticker_list)

In [14]:
def get_commission_dict(tickers: list):
    commission_dict = {}
    for ticker in tickers:
        commission_dict[ticker] = (transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Брокера'] + \
                                     transactions_executed.loc[transactions_executed['Код'] == ticker, 'Комиссия Биржи']) \
                                    .sum().round(2)
    return commission_dict

In [15]:
commission_dict = get_commission_dict(ticker_list)

In [16]:
def get_coupon_dict(tickers: list):
    coupon_dict = {}
    url = "https://iss.moex.com/iss/engines/stock/markets/bonds/boards/TQOB/securities.csv?iss.meta=off&iss.only" \
          "=securities&securities.columns=SECID,ACCRUEDINT "
    csv_text = requests.get(url).text.split('\n')

    for ticker in tickers:
        for line in csv_text:
            line = line.split(';')
            if ticker in line:
                coupon_dict[ticker] = line[1]
    return coupon_dict

In [17]:
coupon_dict = get_coupon_dict(ticker_list)

In [18]:
portfolio_dict = {
    'Котировки': last_prices_dict,
    'НКД': coupon_dict,
    'Количество': share_amount_dict,
    'Средняя цена': share_price_dict,
    'Комиссия': commission_dict
}

main = pd.DataFrame.from_dict(portfolio_dict)
main.index.name = 'Название'
main.loc[main['Котировки'] == '', 'Котировки'] = np.nan
main.dropna(axis=0, subset='Котировки', inplace=True)
main[['Котировки', 'НКД']] = main[['Котировки', 'НКД']].astype('float64')
# main.loc['SU25084RMFS3', 'Котировки'] = main.loc['SU25084RMFS3', 'Котировки'] * 10
# main.loc['SU25084RMFS3', 'Текущая цена'] = (main.loc['SU25084RMFS3', 'Котировки'] + main.loc['SU25084RMFS3', 'НКД']) * main.loc['SU25084RMFS3', 'Количество']
main['Текущая цена'] = main['Котировки'] * main['Количество']
main['P/L, руб.'] = main['Текущая цена'] - main['Средняя цена'] * main['Количество']
main['P/L, %'] = (main['Котировки'] * 100 / main['Средняя цена'] - 100).round(2)

In [19]:
bond_price, bond_NKD, bond_amount, bond_average_price, bond_commission = \
            main.loc['SU25084RMFS3', 'Котировки'], \
            main.loc['SU25084RMFS3', 'НКД'], \
            main.loc['SU25084RMFS3', 'Количество'], \
            main.loc['SU25084RMFS3', 'Средняя цена'], \
            main.loc['SU25084RMFS3', 'Комиссия']

main.loc['SU25084RMFS3', 'Котировки'] = bond_price * 10
main.loc['SU25084RMFS3', 'Текущая цена'] = (bond_price * 10 + bond_NKD) * bond_amount
main.loc['SU25084RMFS3', 'P/L, руб.'] = main.loc['SU25084RMFS3', 'Текущая цена'] - bond_average_price * bond_amount
main.loc['SU25084RMFS3', 'P/L, %'] = (((bond_price * 10 + bond_NKD) * 100 / bond_average_price) - 100).round(2)

In [20]:
main

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBMX,17.466,NaN,830,15.7570,4.98,14496.78,1418.47,10.85
SBER,257.870,NaN,40,224.6600,8.06,10314.80,1328.40,14.78
MTSS,281.000,NaN,20,300.9000,5.41,5620.00,-398.00,-6.61
GAZP,173.770,NaN,30,174.1700,4.71,5213.10,-12.00,-0.23
LKOH,6140.500,NaN,1,4821.3300,4.33,6140.50,1319.17,27.36
MGNT,5551.000,NaN,1,4186.2600,3.76,5551.00,1364.74,32.60
YNDX,2568.800,NaN,1,2642.3800,2.38,2568.80,-73.58,-2.78
CHMF,1287.200,NaN,2,1299.8700,2.34,2574.40,-25.34,-0.97
AFKS,17.399,NaN,200,17.9420,3.23,3479.80,-108.60,-3.03


In [21]:
main.to_csv('portfolio_main.csv', index_label=False)

In [22]:
df = pd.read_csv('portfolio_main.csv')
df.index.name = 'Название'
df

,Котировки,НКД,Количество,Средняя цена,Комиссия,Текущая цена,"P/L, руб.","P/L, %"
Название,,,,,,,,
SBMX,17.466,NaN,830,15.7570,4.98,14496.78,1418.47,10.85
SBER,257.870,NaN,40,224.6600,8.06,10314.80,1328.40,14.78
MTSS,281.000,NaN,20,300.9000,5.41,5620.00,-398.00,-6.61
GAZP,173.770,NaN,30,174.1700,4.71,5213.10,-12.00,-0.23
LKOH,6140.500,NaN,1,4821.3300,4.33,6140.50,1319.17,27.36
MGNT,5551.000,NaN,1,4186.2600,3.76,5551.00,1364.74,32.60
YNDX,2568.800,NaN,1,2642.3800,2.38,2568.80,-73.58,-2.78
CHMF,1287.200,NaN,2,1299.8700,2.34,2574.40,-25.34,-0.97
AFKS,17.399,NaN,200,17.9420,3.23,3479.80,-108.60,-3.03
